# Example calculation of slab DCLC

In [ ]:
from __future__ import division, print_function

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import dredge as dr
import dredge.special
import dredge.vdf
from dredge import ESPerp_GradRho_Species

%load_ext autoreload
%autoreload 2

## Plasma parameters

In [ ]:
mi = 2*1.67e-24  # deuterium mass in g
me = 9.11e-28  # electron mass in g
mi_me = mi/me  # ion/electron mass ratio

Ti_Te = 8  # doesnt matter if not using warm electron response

Omci_ompi = 2.530916e-2  # encodes n0, B0
#Omce_ompe = Omci_ompi*mi_me**0.5

epsilon = -1/1.5  # notice negative sign to put ion diamagnetic drift along +k
bessel_nmax = 20

In [ ]:
vperp = np.linspace(1e-6, 3, 500)
Freduced = dr.vdf.gerver(vperp, 1, 10)

In [ ]:
plt.figure(figsize=(2,1.5))
plt.plot(vperp, Freduced)
plt.xlabel(r'$v_\perp$')
plt.ylabel('$F(v_\perp)$')
plt.show()

## Dispersion solve grid

The $(k, \Re\omega, \Im\omega)$ grid of shape (301,301,202) uses about 1.7 GB memory

In [ ]:
k_vec = np.linspace(1e-5, 15, 301)

omega_re_vec = np.linspace(1e-5, 3.95, 301)

omega_im_vec = np.linspace(0, 4, 201)
#omega_im_vec = np.concatenate((-1*omega_im_vec[:0:-1], omega_im_vec))

# ensure omega_im_vec=0 is not the "boundary" value
# so that we can identify local minima with zero growth/damping
omega_im_vec = np.insert(omega_im_vec, 0, -1*omega_im_vec[1])

# require one sample point at Im(omega) = zero exactly
# to get undamped normal modes
assert np.any(omega_im_vec == 0.)

## Dispersion solve

In [ ]:
ionh = dr.ESPerp_GradRho_Species(
    ms_m0 = 1,
    qs_q0 = 1,
    Ts_T0 = 1,
    k0_vec = k_vec,
    omega0_re_vec = omega_re_vec,
    omega0_im_vec = omega_im_vec,
)

lec = dr.ESPerp_GradRho_Species(
    ms_m0 = 1./mi_me,
    qs_q0 = -1,
    Ts_T0 = 1./Ti_Te,
    k0_vec = k_vec,
    omega0_re_vec = omega_re_vec,
    omega0_im_vec = omega_im_vec,
)

In [ ]:
ionh.cache_besselJ_integrals(
    bessel_nmax = bessel_nmax,
    Freduced = Freduced,
    vperp = vperp,
    verbose = True,
)

# with_prime=False takes ~15 sec
# with_prime=True takes ~30 sec
ionh.cache_bessel_sums(verbose=True, with_prime=False)

## Compute full dispersion relation and find approximate roots

In [ ]:
chi_i = ionh.chi_perp_kinetic(
    epsilon0 = epsilon,
    ns_n0 = 1,
    omp0_Omc0 = 1./Omci_ompi,
)

chi_e = lec.chi_perp_fluid(
    epsilon0 = epsilon,
    ns_n0 = 1,
    omp0_Omc0 = 1./Omci_ompi,
    warm = False,
)

# dispersion D
dd = 1 + chi_i + chi_e

In [ ]:
inds = ionh.grid_roots(np.abs(dd))
k_root = k_vec[inds[0]]
omega_re_root = omega_re_vec[inds[1]]
omega_im_root = omega_im_vec[inds[2]]

In [ ]:
stable = omega_im_root == 0.
unstab = omega_im_root > 0

## Plot dispersion relation

In [ ]:
plt.figure(figsize=(3.375,2))

plt.plot(k_root[stable], omega_re_root[stable], '.', markersize=1, c='k', label=r'Stable $\Re(\omega)$')
plt.plot(k_root[unstab], omega_re_root[unstab], '-', label=r'Unstable $\Re(\omega)$')
plt.plot(k_root[unstab], omega_im_root[unstab], '-', label=r'Unstable $\Im(\omega)$')

plt.grid()
plt.xlabel(r'$k \rho_\mathrm{Li}$')
plt.ylabel(r'$\omega/\Omega_\mathrm{ci}$')

plt.xlim(0, np.amax(k_vec))
plt.ylim(0, np.amax(omega_re_vec))
plt.legend(ncol=3, loc='upper left', fontsize='x-small')

plt.show()

In [ ]:
k_select = 4
ik = np.searchsorted(k_vec, k_select)

In [ ]:
plt.imshow(
    np.abs(dd[ik,:,:]).T, origin='lower',
    extent=ionh.mesh_extent()[2:],
    interpolation='none',
    norm=mpl.colors.LogNorm(vmin=1e0, vmax=1e3),
    cmap='turbo',
)
plt.gca().set_aspect('auto')
plt.xlabel(r'$\Re(\omega)$')
plt.ylabel(r'$\Im(\omega)$')
plt.colorbar()
plt.title(r'$|\mathcal{{D}}(k={:.2f})|$'.format(k_vec[ik]))
plt.show()